In [2]:
import pdfplumber
import pandas as pd
import re
import string

In [3]:
text = []
with pdfplumber.open('./xxxx.pdf') as pdf:
    for i in range(len(pdf.pages)):
        first_page = pdf.pages[i]
        text.append(first_page.extract_text())
        
# text
new_text = text[91:]

a=  []
for i in new_text:
    if i == None:
        continue
    else:
        a.append(i)
new_text_join = ''.join(a)

jie_text = re.split('(第.+节.+\n)',new_text_join)
jie_name = re.findall('(第.+节.+\n)',new_text_join)

miss = [jie_name[1]]+[jie_name[33]]+[jie_name[89]]+[jie_name[92]]+[jie_name[101]]+[jie_name[132]]
# jie_name.pop(30)
for i in jie_name:
    if '概 论' in i:
        jie_name.remove(i) 
    for j in miss:
        if j in i:   
            jie_name.remove(i)

fuhao_s = ('（', '【', '[')
fuhao_e = ('】', '）', ']')

disease_text = []
for i in jie_name:
    jie_index = jie_text.index(i)
    if jie_index < len(jie_text):
        jie_describe = jie_text[jie_index+1]
        disease_text.append(jie_describe)
data = pd.DataFrame({'disease':jie_name,'text':disease_text})
data['text_normal'] = ''

reg = "[^0-9A-Za-z\u4e00-\u9fa5]"

for i,row in data.iterrows():
    disease_text = row['text']
    text_dict = {}
    text_dict['描述'] = []
    word = re.split(r'(（.+】|【.+）|［.+］|【.+】|【.+］|[.+】|【.+])',disease_text)    
    for j in word:
        key = re.sub(reg,'',j)
        j_index = word.index(j)
        if j_index < len(word):
            if j_index >= 1:
                if j.startswith(fuhao_s) and j.endswith(fuhao_e):
                    if j in text_dict:                       
                        text_dict[key] += word[word.index(j)+1]
                    else:
                        text_dict[key] = word[j_index+1]
                else:
                    continue 
            else:
                text_dict.get('描述').append(word[j_index])
        else:
            text_dict.get('描述').append(word[j_index])
    data.loc[data['disease']==row['disease'],'text_normal'] = [text_dict]
signtext = string.punctuation + '\n'+ '。'+'）'+'“'+'”'+'，'+'O'+'0'
data['disease'] = data['disease'].apply(lambda x:x.translate(str.maketrans('', '', signtext))).apply(lambda x:x.replace(' ','')).apply(lambda x:re.findall('(?<=节).*',x)).apply(lambda x: ",".join(x) if isinstance(x,list) else x)
data

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\disease_pdf\\医学\\参考资料\\医学教材\\《妇产科学》人卫第9版教材--高清彩色.pdf'